In [8]:
import numpy as np
from numpy.random import random_integers as rand
import pyquil
from pyquil.quil import Program
from pyquil.api import QVMConnection
from pyquil.gates import *
from scipy import linalg

qvm = QVMConnection()

In [13]:
def initialise(prog):
    init_gate = np.identity(2**3)
    for i in [2,3,4,5]:
        init_gate[i,i] = 0.
    init_gate[2,3] = 1.
    init_gate[3,2] = 1.
    init_gate[4,5] = 1.
    init_gate[5,4] = 1.


    prog.defgate("INIT", init_gate)
    prog += H(10)
    prog += H(1) 
    prog.inst(("INIT",10,1,6))
    
    return prog

In [10]:
def maze():
    prog = Program()
    prog = initialise(prog)
    print(qvm.wavefunction(prog))
    #prog.measure(5,5)
    #print(prog)
    return prog
maze()

(0.5+0j)|000> + (0.5+0j)|011> + (0.5+0j)|101> + (0.5+0j)|110>


In [14]:
from grove.tomography.state_tomography import do_state_tomography
from grove.tomography.utils import notebook_mode
from pyquil.api import QPUConnection, get_devices, Job
import json
from mock import MagicMock
from __future__ import print_function
import matplotlib.pyplot as plt

qubits = [10,1,6]
qvm = QVMConnection()
NUM_SAMPLES=2000
# QPU
#online_devices = [d for d in get_devices() if d.is_online()]
#if online_devices:
#    d = online_devices[0]
#    qpu = QPUConnection(d.name)
#    print("Found online device {}, making QPUConnection".format(d.name))
#else:
#    qpu = QVMConnection()
#    print("Could not find online device, defaulting to QVM")

In [12]:
#Specify QPU
acorn = get_devices(as_dict=True)['19Q-Acorn']
qpu = QPUConnection(acorn)

In [6]:
import os
def plot_state():

    prog = maze()
    print("Running state tomography on the QPU...")
    state_tomography_qpu, _, _ = do_state_tomography(prog, NUM_SAMPLES, qpu, qubits)
    print("State tomography completed.")
    print("Running state tomography on the QVM for reference...")
    state_tomography_qvm, _, _ = do_state_tomography(prog, NUM_SAMPLES, qvm, qubits)
    print("State tomography completed.")
    print(state_tomography_qvm.rho_est)
    state_fidelity = state_tomography_qpu.fidelity(state_tomography_qvm.rho_est)

    print(state_tomography_qpu.rho_est)
    qpu_plot = state_tomography_qpu.plot()
    qpu_plot.text(0.35, 0.9, r'$Fidelity={:1.1f}\%$'.format(state_fidelity*100), size=20)
    #plt.show()
    plt.savefig(os.path.join("state_tomo","state_tomography_qpu_initial.eps"))
    state_tomography_qvm.plot()
    #plt.show()
    plt.savefig(os.path.join("state_tomo","state_tomography_theory_initial.eps"))
  

In [15]:
plot_state()

  0%|          | 0/8 [00:00<?, ?it/s]

(0.5+0j)|000> + (0.5+0j)|011> + (0.5+0j)|101> + (0.5+0j)|110>
Running state tomography on the QPU...


DeviceRetuningError: Error: device 19Q-Acorn is temporarily offline for retuning

The device you requested is temporarily down for retuning. Use the following
code to check for the currently available devices:

    from pyquil.api import get_devices
    print(get_devices())